In [20]:
import pandas as pd
import numpy as np
from scipy.sparse import csr_matrix
import seaborn as sns

In [30]:
df = pd.read_csv("../Assignment/Module8_AssoicationRuleMining/BX-Book-Ratings.csv", encoding='latin-1' )
df.head()

,user_id,isbn,rating
0,276725,034545104X,0
1,276726,155061224,5
2,276727,446520802,0
3,276729,052165615X,3
4,276729,521795028,6


In [31]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1048575 entries, 0 to 1048574
Data columns (total 3 columns):
 #   Column   Non-Null Count    Dtype 
---  ------   --------------    ----- 
 0   user_id  1048575 non-null  int64 
 1   isbn     1048575 non-null  object
 2   rating   1048575 non-null  int64 
dtypes: int64(2), object(1)
memory usage: 24.0+ MB


In [64]:
# Find how many people given rating for each book
totalRating_df = df.groupby('isbn')["rating"].count().reset_index().rename(columns= {"rating": "totalRating"})
totalRating_df

,isbn,totalRating
0,904492401X,1
1,#069580216X,1
2,#6612432,1
3,(THEWINDMILLP,1
4,)416195113,1
...,...,...
322097,`3502103682,2
322098,b00005wz75,1
322099,cn108465,1
322100,cn113107,1


In [68]:
totalRating_df = totalRating_df.sort_values("totalRating", ascending= False)
totalRating_df

,isbn,totalRating
317060,971880107,2264
86839,316666343,1164
135465,385504209,813
80725,312195516,668
200405,60928336,662
...,...,...
228642,696021137,1
257919,808509519,1
272998,8423313174,1
228644,696022214,1


In [73]:
df_merged = df.merge(totalRating_df, on = "isbn")
df_merged.sort_values("totalRating", ascending= False)

,user_id,isbn,rating,totalRating
10803,154125,971880107,0,2264
10519,122988,971880107,0,2264
10521,123168,971880107,0,2264
10522,123182,971880107,0,2264
10523,123501,971880107,0,2264
...,...,...,...,...
649058,31639,689848048,0,1
649057,31632,747406758,5,1
899963,118275,786500700,0,1
248424,2358,044175046X,0,1


In [83]:
## Lets see how many people rating for each book.
df_merged["rating"].value_counts()
# So 0 is rated majortity, this could be people not rated as well. Lets ignore this data.

0     651330
8      95037
10     71814
7      70018
9      62101
5      46953
6      33753
4       8031
3       5445
2       2492
1       1601
Name: rating, dtype: int64

# Take only books with more than 0 rating

In [88]:
df_merged = df_merged[df_merged["rating"] > 0]
df_merged

,user_id,isbn,rating,totalRating
1,2313,034545104X,5,53
3,8680,034545104X,5,53
4,10314,034545104X,9,53
9,50403,034545104X,9,53
13,63970,034545104X,8,53
...,...,...,...,...
1048543,250734,1891620495,10,1
1048556,250750,553527517,5,1
1048557,250750,970349505,4,1
1048558,250750,1552790207,8,1


# Sample 1000 data from the dataset for simplicity

In [161]:
data = df_merged.sample(1000)
data

,user_id,isbn,rating,totalRating
598395,25409,582330866,10,1
1014368,212595,330347683,5,1
328915,102129,1551668459,8,47
553434,17950,140089179,7,5
542731,43698,679781552,8,4
...,...,...,...,...
468745,178201,688171621,9,16
526651,15576,684865866,8,1
708443,239749,3596154049,4,16
573028,21208,892964472,7,7


In [162]:
data.reset_index(inplace=True)
data

,index,user_id,isbn,rating,totalRating
0,598395,25409,582330866,10,1
1,1014368,212595,330347683,5,1
2,328915,102129,1551668459,8,47
3,553434,17950,140089179,7,5
4,542731,43698,679781552,8,4
...,...,...,...,...,...
995,468745,178201,688171621,9,16
996,526651,15576,684865866,8,1
997,708443,239749,3596154049,4,16
998,573028,21208,892964472,7,7


# Prepare the pivot table

In [163]:
pivot_data = data.pivot_table(values="rating",
                                index="isbn",
                                columns="user_id").fillna(0)
pivot_data

user_id,230,243,254,472,1131,1424,1807,2506,2850,2896,...,250184,250344,250367,250398,250543,250683,276847,276872,278346,278418
isbn,,,,,,,,,,,,,,,,,,,,,
000648381X,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
000649840X,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
006017143X,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
006019491X,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,8.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
009942276X,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9727113192,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9728608128,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
983124043X,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [164]:
## Convert Pivot to spare matrix
data_csr = csr_matrix (pivot_data)
data_csr.toarray()

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

# Train the model with pivot table

In [165]:
from sklearn.neighbors import NearestNeighbors

In [166]:
knn_model = NearestNeighbors(metric= "cosine", algorithm = "brute")
knn_model.fit(data_csr)

NearestNeighbors(algorithm='brute', metric='cosine')

# Randomly select the book from the pivot table, for recommendation

In [167]:
random = np.random.choice(pivot_data.shape[0])
random

297

In [168]:
distance, indices = knn_model.kneighbors(pivot_data.iloc[random].values.reshape(1, -1), n_neighbors = 5)

In [169]:
distance, indices

(array([[0., 1., 1., 1., 1.]]),
 array([[297, 654, 653, 655, 651]], dtype=int64))

In [170]:
for each in range(0, len(distance.flatten())):
    if each == 0:
        print("Current Reading book is {0}:".format(pivot_data.index[indices.flatten()][each]))
    else:
        print("Recommended book  #isbn: {0}".format(pivot_data.index[indices.flatten()][each]))

Current Reading book is 3499114372:
Recommended book  #isbn: 60195886
Recommended book  #isbn: 60195703
Recommended book  #isbn: 60196246
Recommended book  #isbn: 60191791


## Now "BookRent" can recommend these books that are close to the current book read by the customer to keep customer continue renewal and enjoy new books